# AI-Assisted Programming with Langchain
> Creating full-featured apps

We've already talked about general programming, transformers, and openai. Let's see where this fits in.

# AI Ecosystem

# Langchain

[Overview of Langchain](https://python.langchain.com/docs/get_started/introduction)

## Creating and Prompting an LLM

In [ ]:
!pip install gradio langchain
!pip install pypdf chromadb transformers
!pip install duckduckgo-search

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
import os

from free_access_models import ChatMistralLocal, LLMMistralLocal, MistralEmbeddingsLocal
import numpy as np

In [ ]:
# auth replicated here for reference just in case you choose to do something similar
#from google.colab import userdata
#os.environ['HUGGINGFACEHUB_API_TOKEN'] = userdata.get('HUGGINGFACEHUB_API_TOKEN')
#os.environ['HF_TOKEN'] = os.environ['HUGGINGFACEHUB_API_TOKEN']

# Basic Prompt Chains

See [Prompt+LLM](https://python.langchain.com/docs/expression_language/cookbook/prompt_llm_parser) for more information

In [ ]:
# basic usage


In [ ]:
# Observe what the prompt looks like when we substitute words into it


In [ ]:
# Now, actually call the entire chain on it


## Even more simplified prompt chains

In [ ]:
# Create total user prompt chain
prompt = ChatPromptTemplate.from_template("")
chain = prompt | model

In [ ]:
# Now, the user can submit literally whatever


# Creating and Using a Knowledge Base : Retrieval Augmented Generation (RAG)

* Conceptual and step-by-step guide about [RAG](https://python.langchain.com/docs/modules/data_connection/)
* Learn more about implementing [RAG](https://python.langchain.com/docs/expression_language/cookbook/retrieval)

In [ ]:
from operator import itemgetter

from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from free_access_models import MistralEmbeddingsLocal

In [ ]:
knowledge_base = 'Pro-Se-Handbook-APPROVED-v2019-2.pdf'

## Document Loaders and Splitters

In [ ]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter



In [ ]:
print(f"Number of Documents: {len(pages)}\n")
pages[:2]

## Vector Stores: A way to store embeddings (hidden states) of your data

## Retrievers: How we select the most relevant data

In [ ]:
# query the vector store
query = 'What forms do I need to fill out to begin with?'

# use a similarity search between the vectors



In [ ]:
# or use the db as a retriever


## RAG
For when we want to actually do generation, but want there to be retrieved documents included

In [ ]:
# Basic question answering template
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""

# compose prompt
prompt = ChatPromptTemplate.from_template(template)

# retriever already created above

In [ ]:
# create the chain


In [ ]:
# get the response


# Agents
Learn more about [Agents](https://python.langchain.com/docs/modules/agents/quick_start) in their Quickstart

### Choose Tools

In [ ]:
# Web search tools
from langchain.tools import DuckDuckGoSearchRun, DuckDuckGoSearchResults
from langchain_community.utilities import DuckDuckGoSearchAPIWrapper

# Creating custom retriever tools
from langchain.tools.retriever import create_retriever_tool

# For tested prompts
from langchain import hub

# For agents
from langchain.agents import create_react_agent
from langchain.agents import AgentExecutor

from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate, MessagesPlaceholder


In [ ]:
# based off of the Guide we used


In [ ]:
# define tools


In [ ]:
# create prompt
prompt = ChatPromptTemplate.from_messages([
    MessagesPlaceholder(variable_name="chat_history", optional=True),
    HumanMessagePromptTemplate.from_template(template='Answer the following questions as best you can or reply with the most meaningful ' +
                                             'response possible. You have access to the following tools:\n\n{tools}\n\n although you can also reply conversationally when appropriate. '
                                             'Use the following format:\n\n' +
                                             'Question: the input question you must answer or statement to which you should reply\n' +
                                             'Thought: you should always think about what to do\n' +
                                             'Action: the action to take, should be one of [{tool_names}]\n'+
                                             'Action Input: the input to the action\n'+
                                             'Observation: the result of the action\n... '+
                                             '(this Thought/Action/Action Input/Observation can repeat up to 2 times)\n'+
                                             'Thought: I now know the final answer\n'+
                                             'Final Answer: the final answer to the original input question or appropriate response is\n\n'+
                                             'Begin!\n\nQuestion or message: {input}\nThought:{agent_scratchpad}'),
  ]
)

In [ ]:
# Create our own llm and agent


In [ ]:
# Execute the agent


## Adding the conversational component

In [ ]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

In [ ]:
# start with empty chat history. We can pass this as a parameter in the invoke


# Create the agent with chat history


In [ ]:
# Start chatting away
agent_with_chat_history.invoke(
    {"input": "I'm going pro se on a civil case against a corporation that scammed me repeatedly. "+
     "How should I start preparing? I'm in the state of Tennesse."},
    config={"configurable": {"session_id": '<something>'}},
)

In [ ]:
agent_with_chat_history.invoke(
    {"input": "Thank you. What do you think will be the most difficult part of representing myself in court?"},
    # This is needed because in most real world scenarios, a session id is needed
    # It isn't really used here because we are using a simple in memory ChatMessageHistory
    config={"configurable": {"session_id": "<something>"}},
)

# Bringing it all together

In [ ]:
import gradio as gr

In [ ]:
def ai_response(message, history):
    agent_message = {'input':message}
    response = agent_with_chat_history.invoke(agent_message, config={"configurable": {"session_id": history}})
    return response['output']

with gr.Blocks() as demo :
    chatbot = gr.Chatbot()
    msg = gr.Textbox()
    clear = gr.ClearButton([msg, chatbot])

    msg.submit(ai_response, [msg, chatbot], [msg, chatbot])

demo.launch()